# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once  a request is received, the API will convert the json encoded request body into the image format. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the images and uses the first function to score them.

In [1]:
import logging
from testing_utilities import img_url_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

import numpy as np
import timeit as t
import base64
import json
from PIL import Image, ImageOps
from io import BytesIO
import logging

number_results = 3
logger = logging.getLogger("model_driver")

def _base64img_to_numpy(base64_img_string):
    decoded_img = base64.b64decode(base64_img_string)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img

def create_scoring_func():
    """ Initialize ResNet 152 Model 
    """   
    start = t.default_timer()
    model = ResNet152(weights='imagenet')
    end = t.default_timer()
    
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(img_array):
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        preds = decode_predictions(preds, top=number_results)[0]       
        return preds
    
    return call_model       

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        base64Dict = json.loads(inputString) 
        for k, v in base64Dict.items():
            img_file_name, base64Img = k, v 
        img_array = _base64img_to_numpy(base64Img)
        preds = scoring_func(img_array)
        resp = {img_file_name: preds}
        responses.append(resp)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return tf.__version__

Writing driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the file driver.py which will bring everything into the context of the notebook.

In [4]:
%run driver.py

Using TensorFlow backend.


We will use the same Lynx image we used ealier to check that our driver works as expected.

In [5]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [6]:
predict_for = get_model_api()

INFO:model_driver:Model loading time: 28443.23 ms


In [7]:
jsonimg = img_url_to_json(IMAGEURL)
json_load_img = json.loads(jsonimg)
body = json_load_img['input']
resp = predict_for(body)

DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'iCCP' 41 292
DEBUG:PIL.PngImagePlugin:iCCP profile name b'ICC Profile'
DEBUG:PIL.PngImagePlugin:Compression method 0
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 345 65536
INFO:model_driver:Predictions: [{'image': [('n02127052', 'lynx', 0.9816487), ('n02128385', 'leopard', 0.007744099), ('n02123159', 'tiger_cat', 0.0036861112)]}]
INFO:model_driver:Predictions took 3110.29 ms


Next, we can move on to [building our docker image](02_BuildImage.ipynb).